## IVX Predictive Regression — Single Predictor Example

This example runs the IVX predictive regression from Kostakis, Magdalinos, and Stamatogiannis (2015) with one predictor for clarity.

The predictive system is

$$
y_t = \mu + A x_{t-1} + e_t
$$

$$
x_t = R x_{t-1} + u_t
$$

with \(R\) diagonal.

### What the script does

* Estimates the predictive regression at a given horizon \(K\)
* Constructs instruments and computes the IVX slope estimate
* Reports the IVX coefficient, individual Wald test, and overall Wald test

### Data

* File: `Macro2021.xlsx`  
* Sheet: `macro`  
* Target variable: `Log_equity_pr`  
* Predictor in this example: `DP` (can be replaced with any other column)  

The data should be in time order. Remove missing values if present.

### How to run

1. Load the Excel file and clean headers to simple names  
2. Keep only `Log_equity_pr` and `DP`  
3. Call `ivxlh(yt, xt, K)` for \(K=1\) and \(K=12\)

The script prints:

* IVX coefficient for the predictor  
* Individual Wald statistic and p-value  
* Overall Wald statistic and p-value  

### Results

When using the same sample and predictor, the results are consistent across implementations for both \(K=1\) and \(K=12\).


In [14]:
# IVX with a single predictor: DP
# File: C:\Users\sampy\Downloads\Macro2021.xlsx | Sheet: "macro"
# Jupyter-ready

import numpy as np
import pandas as pd
from scipy.stats import chi2

def ivxlh(yt, xt, K, printres=False):
    yt = np.asarray(yt).reshape(-1, 1)
    xt = np.asarray(xt)

    xlag = xt[:-1, :]
    xt = xt[1:, :]
    y = yt[1:, :]

    nn, l = xlag.shape
    X = np.column_stack((np.ones(nn), xlag))

    Aols, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    epshat = y - X @ Aols
    Aols = Aols.flatten()

    rn = np.zeros((l, l))
    for i in range(l):
        Xi = xlag[:, i].reshape(-1, 1)
        ri, _, _, _ = np.linalg.lstsq(Xi, xt[:, i], rcond=None)
        rn[i, i] = ri.item()

    u = xt - xlag @ rn
    corrmat = np.corrcoef(np.column_stack((epshat, u)).T)

    covepshat = (epshat.T @ epshat) / nn
    covu = np.zeros((l, l))
    for t in range(nn):
        covu += np.outer(u[t, :], u[t, :])
    covu /= nn

    covuhat = np.array([np.sum(epshat.ravel() * u[:, i]) for i in range(l)]) / nn

    m = int(np.floor(nn ** (1 / 3)))
    uu = np.zeros((l, l))
    for h in range(1, m + 1):
        a = np.zeros((l, l))
        for t in range(h, nn):
            a += np.outer(u[t, :], u[t - h, :])
        uu += (1 - h / (m + 1)) * a
    uu /= nn
    Omegauu = covu + uu + uu.T

    q = np.zeros((m, l))
    for h in range(1, m + 1):
        p = np.zeros((nn - h, l))
        for t in range(h, nn):
            p[t - h, :] = u[t, :] * epshat[t - h, 0]
        q[h - 1, :] = (1 - h / (m + 1)) * np.sum(p, axis=0)
    residue = np.sum(q, axis=0) / nn
    Omegaeu = covuhat + residue

    n = nn - K + 1
    Rz = (1 - 1 / (nn ** 0.95)) * np.eye(l)
    diffx = xt - xlag
    z = np.zeros((nn, l))
    z[0, :] = diffx[0, :]
    for i in range(1, nn):
        z[i, :] = z[i - 1, :] @ Rz + diffx[i, :]

    Z = np.vstack([np.zeros((1, l)), z[:n - 1, :]])
    zz = np.vstack([np.zeros((1, l)), z[:-1, :]])
    ZK = np.zeros((n, l))
    for i in range(n):
        ZK[i, :] = np.sum(zz[i:i + K, :], axis=0)

    yy = np.zeros(n)
    for i in range(n):
        yy[i] = np.sum(y[i:i + K, 0])
    xK = np.zeros((n, l))
    for i in range(n):
        xK[i, :] = np.sum(xlag[i:i + K, :], axis=0)

    meanxK = np.mean(xK, axis=0)
    Yt = yy - np.mean(yy)
    Xt = np.column_stack([xK[:, i] - meanxK[i] for i in range(l)])

    Aivx = Yt @ Z @ np.linalg.pinv(Xt.T @ Z)

    FM = covepshat - (Omegaeu.reshape(1, -1) @ np.linalg.inv(Omegauu) @ Omegaeu.reshape(-1, 1))
    M = ZK.T @ ZK * covepshat.item() - n * np.outer(np.mean(ZK, axis=0), np.mean(ZK, axis=0)) * FM.item()

    Q = np.linalg.pinv(Z.T @ Xt) @ M @ np.linalg.pinv(Xt.T @ Z)

    Wivx_stat = (Aivx.reshape(1, -1) @ np.linalg.pinv(Q) @ Aivx.reshape(-1, 1)).item()
    Wivx_p = 1 - chi2.cdf(Wivx_stat, df=l)
    Wivx = np.array([[Wivx_stat], [Wivx_p]])

    se = np.sqrt(np.diag(Q))
    WivxInd_stat = (Aivx / se) ** 2
    WivxInd_p = 1 - chi2.cdf(WivxInd_stat, df=1)
    WivxInd = np.vstack([WivxInd_stat, WivxInd_p])

    if printres:
        print(f"Horizon K = {K}")
        print("IVX slopes:", Aivx)
        print("Overall Wald [stat, p]:", Wivx.ravel())

    return Aols, Aivx, Wivx, WivxInd, Q, corrmat

# ---- Load Excel ----
path = r"C:\Users\sampy\Downloads\Macro2021.xlsx"
df = pd.read_excel(path, sheet_name="macro")

# Clean column names like before
df.columns = (
    df.columns.astype(str).str.strip()
      .str.replace(r"\s+\(.*\)", "", regex=True)
      .str.replace("%", "", regex=False)
      .str.replace(" ", "_")
)

# Keep date order if present
if "date" in df.columns:
    df = df.sort_values("date")

# Use only DP as predictor
y_col = "Log_equity_pr"
x_cols = ["TBL"]

use = df[[y_col] + x_cols].dropna().copy()

yt = use[y_col].to_numpy()
xt = use[x_cols].to_numpy()  # shape (T,1)

# ---- Run IVX for K=1 and K=12 with DP only ----
Aols_1, Aivx_1, Wivx_1, WivxInd_1, Q_1, corr_1 = ivxlh(yt, xt, K=1, printres=True)
Aols_12, Aivx_12, Wivx_12, WivxInd_12, Q_12, corr_12 = ivxlh(yt, xt, K=12, printres=True)

# Simple output
res_k1 = pd.DataFrame({
    "predictor": x_cols,
    "ivx_coef": np.asarray(Aivx_1).ravel(),
    "wald_stat": WivxInd_1[0, :],
    "p_value": WivxInd_1[1, :]
})
res_k12 = pd.DataFrame({
    "predictor": x_cols,
    "ivx_coef": np.asarray(Aivx_12).ravel(),
    "wald_stat": WivxInd_12[0, :],
    "p_value": WivxInd_12[1, :]
})

print("\nK=1 — overall:", float(Wivx_1[0,0]), float(Wivx_1[1,0]))
display(res_k1)

print("\nK=12 — overall:", float(Wivx_12[0,0]), float(Wivx_12[1,0]))
display(res_k12)


Horizon K = 1
IVX slopes: [-0.00094376]
Overall Wald [stat, p]: [3.17207    0.07490745]
Horizon K = 12
IVX slopes: [-0.00075034]
Overall Wald [stat, p]: [1.88055615 0.17027095]

K=1 — overall: 3.1720699989933996 0.07490745188493053


,predictor,ivx_coef,wald_stat,p_value
0,TBL,-0.000944,3.17207,0.074907



K=12 — overall: 1.880556146264193 0.17027095155317284


,predictor,ivx_coef,wald_stat,p_value
0,TBL,-0.00075,1.880556,0.170271
